# Chat History Inital EDA (17/07/24)

**Intro** 

* This notebook is exploring the chat history logs for Redbox pulled on 17th July. 

* Populated columns of this data are 'id', 'created_at', 'modified_at', 'name', 'users', 'id.1', 'created_at.1', 'modified_at.1', 'chat_history', 'text', 'role'.

* Aim of this work is to discover how users are using Redbox and find any other features that may be important to log.

* **IMPORTANT** - For this to work you must save your chathistory.csv dump in notebooks/evaluation/data/chat_histories

**Next Steps** 

* Want to get the actual Redbox route so we don't have to rely on inferred data

* *Real* patterns hard to find due to limited data

* Could be interested in how different LLM change user behaviour - would need to store this

* Limitations right now are that we can't really tell whether a chat is successful (e.g. User gets intended output from Redbox). Creating a UI and measuring this would mean we could identify differences in user behaviour that lead to useful/unuseful outputs.

* For RAG use, I'd be interested to see what documents work well or not. Maybe we don't want to store the whole document but maybe the size, type etc. Pairing this with a success variable could lead to interesting insights. 

In [ ]:
import pandas as pd

In [ ]:
chathistory_df = pd.read_csv('evaluation/data/chat_histories/chathistory.csv')

In [ ]:
chathistory_df.columns

In [ ]:
chathistory_df.info()

In [ ]:
chathistory_df.head()

In [ ]:
chathistory_df['role'].value_counts()

In [ ]:
# Ensuring date columns are stored as datetimes
date_cols = ['created_at', 'modified_at','created_at.1', 'modified_at.1']
for i in date_cols:
       chathistory_df[i] = pd.to_datetime(chathistory_df[i])
chathistory_df.info()

## Top users (inc. over time)

In [ ]:
# Pulling out user input rows only
# Identifying top users of redbox
user_inputs_df = chathistory_df[(chathistory_df['role']=='user')]
user_inputs_df['users'].value_counts()

In [ ]:
# How has total redbox usage changed overtime
user_inputs_df.groupby(chathistory_df['modified_at'].rename('Days').dt.day).size().plot(ylabel='No. of prompts', title='Total Redbox Usage Over Time')

This shows a general increase in usage of redbox. 

Now, interested in increase/decrease by user.

In [ ]:
import seaborn as sns

# Plotting how each users usage has changed over time
pivot_user_time = user_inputs_df[['modified_at', 'users']].groupby(pd.Grouper(key='modified_at', axis=0, freq='2D', sort=True)).value_counts().reset_index(name='count').pivot(index='modified_at', columns='users', values='count')
fig = sns.lineplot(data=pivot_user_time, markers=True)
fig.set_xlabel('Date')
fig.set_ylabel('No. of Prompts')
fig.tick_params(axis='x', rotation=90)
sns.move_legend(fig, "upper left", bbox_to_anchor=(1, 1), title='Users')
fig.set_title('Line graph to show number of prompts for each user over time')

Maybe not the best plot to use due to null entries, however, I think it shows trends in users usage and shows those users which have only used Redbox once.

Next, interested to see whether users that aren't using it as frequently are using certain routes of Redbox to those who are, for example. 

Routes column is not currently populated so I have inferred this by identifying whether the user has used @chat, @summarise or otherwise.

## Popularity of User Routes (inc. process map)

In [ ]:
# Creating the inferred routes from the text column 
def routes(row):
    if '@chat' in row['text']:
        val = 'chat'
    elif '@summarise' in row['text']:
        val = 'summarise'
    else:
        val = 'rag'
    return val

user_inputs_df['infer_route'] = user_inputs_df.apply(routes, axis=1)
user_inputs_df.head()

In [ ]:
# Creating df of each users most popular prompt route
popular_routes = user_inputs_df[['users', 'infer_route']].value_counts().reset_index(name='count')
popular_routes

In [ ]:
# sns.barplot(popular_routes['infer_route'].value_counts().reset_index(name='count'), x='infer_route', y='count', hue='infer_route', title='Most popular Routes')
popular_routes[['infer_route', 'count']].groupby('infer_route').sum().reset_index()
fig = sns.barplot(popular_routes[['infer_route', 'count']].groupby('infer_route').sum().reset_index(), x='infer_route', y='count', hue='infer_route')
fig.set_xlabel('Inferred Route')
fig.set_title('Bar plot to show most popular routes')
fig.set_ylabel('No. of Prompts')

In [ ]:
fig = sns.barplot(data=popular_routes, x='users', y='count', hue='infer_route')
fig.tick_params(axis='x', rotation=90)
fig.set_xlabel('Users')
fig.set_title('Bar plot to show most popular routes by user')
fig.set_ylabel('No. of Prompts')
fig.legend(title='Inferred Route')

In [ ]:
# See if theres a change in popular routes over time
route_over_time = user_inputs_df[['modified_at', 'infer_route']].groupby(pd.Grouper(key='modified_at', axis=0, freq='2D', sort=True)).value_counts().reset_index(name='count')
route_over_time

In [ ]:
fig = sns.lineplot(data=route_over_time, markers=True, x='modified_at', y='count', hue='infer_route')
fig.set_ylabel('No. of Prompts')
fig.set_xlabel('Date')
fig.tick_params(axis='x', rotation=90)
fig.legend(title='Inferred Route')
sns.move_legend(fig, "upper left", bbox_to_anchor=(1, 1))
fig.set_title('Line graph to show number of inputs for each prompt type over time')



Shows that over the last couple of days, the chat function has overtaken summarise in terms of popularity. 

### Process Map 

Interested to see if theres any distinctive patterns between users, whether that effects their use of Redbox. It would be interesting to compare successful and unsuccessful routes if we measure this somehow so we can provide users with better training.

In [ ]:
from pm4py.objects.conversion.log import converter as log_conversion_factory
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py.visualization.heuristics_net import visualizer as hn_visualizer

def create_hnet_vis(event_df):

    # Convert DataFrame to pm4py event log format
    event_df.columns = ['case:concept:name', 'time:timestamp', 'concept:name']
    log = log_conversion_factory.apply(event_df)

    # Discover Heuristic Net
    heu_net = heuristics_miner.apply_heu(log)

    # Visualize Heuristic Net
    gviz = hn_visualizer.apply(heu_net)
    hn_visualizer.view(gviz)


route_map_df = user_inputs_df[['id', 'modified_at', 'infer_route']]
create_hnet_vis(route_map_df)

In [ ]:
# Creating a process map for each user.
for i in user_inputs_df['users'].unique():
    user_input_df_filter = user_inputs_df[user_inputs_df['users']==i]
    route_map_df = user_input_df_filter[['id', 'modified_at', 'infer_route']]
    print(i)
    create_hnet_vis(route_map_df)

## Prompt detail effects

I'm assuming that each chat ends 'successfully' (quite flawed) but want to see whether the level of detail (currently measuring this by word length but could use more advanced metrics) effects how many prompts a user needs to try before they are done with Redbox. 

In [ ]:
# Creating column for number of words
user_inputs_df['no_input_words'] = user_inputs_df["text"].apply(lambda n: len(n.split()))
user_inputs_df.head()

In [ ]:
fig = sns.displot(user_inputs_df.no_input_words)
fig.set_axis_labels(x_var='No. of words in prompt', y_var='Count')

Seems to be an anomaly over 1400 words, removing this to see distribution of rest.

In [ ]:
sns.histplot(user_inputs_df[user_inputs_df['no_input_words']<700]['no_input_words'])
fig.set_axis_labels(x_var='No. of words in prompt', y_var='Count')

In [ ]:
# How does this vary between users
fig = sns.displot(
    user_inputs_df[user_inputs_df['no_input_words']<700],
    x="no_input_words", 
    col="users", col_wrap=4, height=4,
)
fig.set_axis_labels(x_var='No. of words in prompt', y_var='Count')

In [ ]:
# How does this vary between users and routes
fig = sns.displot(
    user_inputs_df[user_inputs_df['no_input_words']<700],
    x="no_input_words", hue='infer_route',
    col="users", col_wrap=4, height=4,
)
fig.set_axis_labels(x_var='No. of words in prompt', y_var='Count')

In [ ]:
no_inputs_df = user_inputs_df[["id", "users"]].groupby("id").value_counts().reset_index(name='no_inputs')
no_inputs_df.head()

In [ ]:
mean_inputs_df = user_inputs_df[["id", "users", "no_input_words"]].groupby(by=['id', 'users']).agg({'no_input_words': 'mean'}).rename(columns={'no_input_words':'mean_input_words'}).reset_index()
mean_inputs_df.head()

In [ ]:
compare_inputs_words_df = no_inputs_df.merge(mean_inputs_df, left_on=['id', 'users'], right_on=['id', 'users'])

# Remove anomaly to see if pattern more clearly - maybe should have been done before calculating the mean
fig = sns.scatterplot(
    data=compare_inputs_words_df[compare_inputs_words_df['mean_input_words']<200],
    x='no_inputs',
    y='mean_input_words',
    hue='users')
fig.set_xlabel('No. of prompts')
fig.set_ylabel('Mean length of prompt')
fig.set_title('Scatter plot comparing number of prompts with the length of prompt for each user session')
sns.move_legend(fig, "upper left", bbox_to_anchor=(1, 1))

In [ ]:
# A very weak negative col, potentially showing the more detailed input a user gives the less inputs they have to use in a session
compare_inputs_words_df[compare_inputs_words_df['mean_input_words']<200][['mean_input_words', 'no_inputs']].corr()

In [ ]:
compare_inputs_words_df[compare_inputs_words_df['mean_input_words']<200].groupby('users')[['mean_input_words', 'no_inputs']].corr()